# Ingest Content

## Notes

Collection of notes to help ingest content from a domain. Includes specific instructions to filter output for some domains (like bradfrost.com).

### bradfrost.com

- Exclude `/blog/link/*` and `/blog/tag/*`

### danmall.com

- Exclude `/topics/*`

### primer.style

- Exclude `/octicons/*` and `/design/foundations/icons/*` and `/view-components/lookbook/*`

### digitaldesign.vattenfall.com

- Exclude `/components/modules/*`.

### volkswagen.frontify.com

- Exclude `/builder/groupui/*`.

## Dependencies

Install the following dependencies first:

In [ ]:
!pip install bs4 jsonlines pytest-playwright

## Install needed browsers

In this case we'll just use Chrome... but Webkit and Firefox get installed, too.

In [ ]:
!playwright install

## Set the domain to ingest

Set the domain you want to have scanned, this will include all subpages on that domain (and that domain only). Excludes links with parameters (`?`) and anchors (`#`).

Also set a name that will be used to create files to persist ingested content.

In [82]:
name = 'volkswagen_frontify_com'
url = 'https://volkswagen.frontify.com/d/rzB71PwpjXgt/'

Think about ways of including a script / snippet per domain to handle edge cases. Some kind of hook?

## Find all internal URLs

First step is to crawl a domain for all internal links leading to HTML content. Do this until everything is discovered. Enter your domain through adjusting the `url` in `__main__`.

Write the set of discovered URLs from `all_links` to disk, converting them to `jsonl` format for easier processing in the next steps. We'll build upon that `page` dict in the following steps.

Scroll delay (currently 5 * .5s) serves as a kind of "backoff" as to not become blocked for DDoS / obvious scraping for the moment.

TODO:
- Still some duplicate URLs (/ vs non-/)
- Not finding some URLs when behind tabbed content? (e.g. https://atlassian.design/components/button/examples -> Usage tab)
- Add exclude list / filter to processed urls (e.g. to exclude stuff like "blog/tag/*")
- Attempt to handle Cookies / Cookie Consent, try to close cookie banners? (e.g. for screenshots)
- Some types of links seem still to fail (e.g. "https://danmall.com/topics/artificial intelligence", as logged)
- Some links (probably because of weird endless scrolling) seem to always time out, like this one: https://www.designbetter.co/principles-of-product-design/break-black-box
- Subdomains seem to have problems (like here: https://design-system.service.gov.uk/ / https://hds.hel.fi/)
- Sometimes empy URLs
- Timeout on https://primer.style/design/foundations/css-utilities/animations
- Retry-Logic and incremental crawling

In [78]:
import asyncio
import requests
import jsonlines
import pathlib
import time
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from slugify import slugify
 
def get_domain(url):
    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return domain

def get_path(url):
    parsed_uri = urlparse(url)
    return parsed_uri.path
 
def get_links(url, content):
    soup = BeautifulSoup(content, 'html.parser')
    domain = get_domain(url)
    links = set()
    for link in soup.find_all('a'):
        link_url = link.get('href')
        if link_url:
            absolute_link = urljoin(url, link_url)
            if absolute_link.startswith(domain):
                links.add(absolute_link)
    return links
 
async def playwright(url, slug):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        screenshot_dir = 'screenshots/' + name
        pathlib.Path(screenshot_dir).mkdir(parents=True, exist_ok=True)
        await page.screenshot(path=f'{screenshot_dir}/{slug}-{p.chromium.name}.png')
        
        for i in range(5):
            await page.mouse.wheel(0, 15000)
            time.sleep(0.500)
            i += 1
        
        content = await page.content()
        await browser.close()
        return content
    
if __name__ == '__main__':
    queue = [url]
    visited = set()
    all_links = set()
    pages = []
 
    while queue:
        url = queue.pop(0)
        visited.add(url)
        response = requests.get(url)
        if (response.ok and "text/html" in response.headers['Content-Type']):
            page = dict()
            print(url)
            if (url.rstrip("/") not in all_links):
                all_links.add(url.rstrip("/"))
                page['url'] = url
                page['slug'] = slugify(get_path(page['url']))
                page['content'] = dict()
                page['content']['html'] = await playwright(page['url'], page['slug'])
                pages.append(page)
    
                links = get_links(page['url'], page['content']['html'])
                for link in links:
                    if link not in visited and link not in queue and '#' not in link and '?' not in link:
                        queue.append(link)
    
    print()
    print('All done! ' + str(len(all_links)) + ' links discovered.')

    with jsonlines.open('pages-' + name + '.jsonl', 'w') as writer:
        writer.write_all(pages)

https://orbit.kiwi/
https://orbit.kiwi/components
https://orbit.kiwi/foundation
https://orbit.kiwi/foundation/design-tokens/
https://orbit.kiwi/kiwi-use/content/grammar-and-mechanics/
https://orbit.kiwi/getting-started/
https://orbit.kiwi/component-status/
https://orbit.kiwi/kiwi-use/content/voice-and-tone/
https://orbit.kiwi/changelog/
https://orbit.kiwi/kiwi-use/content/
https://orbit.kiwi/foundation/color/
https://orbit.kiwi/foundation/spacing
https://orbit.kiwi/components/
https://orbit.kiwi/foundation/
https://orbit.kiwi/foundation/accessibility/
https://orbit.kiwi/design-patterns/
https://orbit.kiwi/development/guides/
https://orbit.kiwi/kiwi-use/content/glossary/
https://orbit.kiwi/foundation/typography
https://orbit.kiwi/themer/
https://orbit.kiwi/foundation/spacing/
https://orbit.kiwi/components/interaction/slider/
https://orbit.kiwi/development/hooks/useintersect/
https://orbit.kiwi/development/guides/developing-layouts/
https://orbit.kiwi/kiwi-use/content/social-media/
https

https://orbit.kiwi/components/input/checkbox/react/
https://orbit.kiwi/components/visuals/icon/react/
https://orbit.kiwi/components/input/choicegroup/react/
https://orbit.kiwi/guides/using-button-type-components/
https://orbit.kiwi/components/action/textlink/react/
https://orbit.kiwi/components/layout/horizontalscroll/react/
https://orbit.kiwi/components/navigation/pagination/react/
https://orbit.kiwi/components/progress-indicators/loading/react/
https://orbit.kiwi/components/primitives/badgeprimitive/react/
https://orbit.kiwi/components/visuals/calloutbanner/react/
https://orbit.kiwi/components/utility/truncate/react/
https://orbit.kiwi/getting-started/support/team/
https://orbit.kiwi/getting-started/support/open-source/
https://orbit.kiwi/design-patterns/progressive-disclosure
https://orbit.kiwi/components/interaction/collapse/react/
https://orbit.kiwi/components/visuals/coupon/react/
https://orbit.kiwi/design-patterns/designing-forms
https://orbit.kiwi/components/action/socialbutton

## More dependencies

Install trafilatura, that will be used to extract the content from pages, and tiktoken to have a first relevant token measurement for complete page content.

In [ ]:
!pip install trafilatura tiktoken

## Extract content from discovered pages

We'll keep Markdown formatting for now. It will be used to split sections from pages by slicing by headlines. 

In [79]:
import re
import jsonlines
import tiktoken
from bs4 import BeautifulSoup
from trafilatura import load_html, extract
from markdown import markdown

enc = tiktoken.encoding_for_model("text-davinci-003")

def markdown_to_text(markdown_string):
    html = markdown(markdown_string)

    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

extracted_content = []
with jsonlines.open('pages-' + name + '.jsonl') as pages:
    for page in pages:
        if (page['content']['html']):
            downloaded = load_html(page['content']['html'])
            parsed = BeautifulSoup(str(downloaded))
            ogTitle = parsed.find("meta", property="og:title")
            title = parsed.title.string if (parsed.title and parsed.title.string) else ogTitle["content"] if ogTitle else page['url']
            result = extract(downloaded, url=page['url'], include_formatting=True)

            if result is None:
                print('couldnt extract:', page['url'])
            else:
                augmented = dict()
                augmented['url'] = page['url']
                augmented['slug'] = page['slug']

                augmented['content'] = page['content']
                augmented['content']['raw'] = markdown_to_text(result)
                augmented['content']['markdown'] = result

                augmented['title'] = title
                augmented['lines'] = result.splitlines()
                augmented['size'] = len(result)
                augmented['token'] = len(enc.encode(result))

                extracted_content.append(augmented)
                print('extracted:', augmented['url'], augmented['title'], str(augmented['token']) + ' Token,', len(result))  

with jsonlines.open('pages-' + name + '_extracted.jsonl', 'w') as pages:
    pages.write_all(extracted_content)
    
print()
print('wrote extracted content to "pages-' + name + '_extracted.jsonl"')

extracted: https://orbit.kiwi/ https://orbit.kiwi/ 223 Token, 1057
extracted: https://orbit.kiwi/components https://orbit.kiwi/components 216 Token, 997
couldnt extract: https://orbit.kiwi/foundation
extracted: https://orbit.kiwi/foundation/design-tokens/ https://orbit.kiwi/foundation/design-tokens/ 3233 Token, 14168
extracted: https://orbit.kiwi/kiwi-use/content/grammar-and-mechanics/ https://orbit.kiwi/kiwi-use/content/grammar-and-mechanics/ 1034 Token, 3894
extracted: https://orbit.kiwi/getting-started/ https://orbit.kiwi/getting-started/ 56 Token, 266
extracted: https://orbit.kiwi/component-status/ https://orbit.kiwi/component-status/ 1166 Token, 3667
extracted: https://orbit.kiwi/kiwi-use/content/voice-and-tone/ https://orbit.kiwi/kiwi-use/content/voice-and-tone/ 603 Token, 2311
extracted: https://orbit.kiwi/changelog/ https://orbit.kiwi/changelog/ 13745 Token, 37414
extracted: https://orbit.kiwi/kiwi-use/content/ https://orbit.kiwi/kiwi-use/content/ 64 Token, 250
extracted: https

extracted: https://orbit.kiwi/components/layout/box/ https://orbit.kiwi/components/layout/box/ 121 Token, 416
extracted: https://orbit.kiwi/components/progress-indicators/emptystate/ https://orbit.kiwi/components/progress-indicators/emptystate/ 413 Token, 1809
extracted: https://orbit.kiwi/components/structure/tile/ https://orbit.kiwi/components/structure/tile/ 307 Token, 1380
extracted: https://orbit.kiwi/components/interaction/stepper/ https://orbit.kiwi/components/interaction/stepper/ 191 Token, 795
extracted: https://orbit.kiwi/components/structure/itinerary/ https://orbit.kiwi/components/structure/itinerary/ 93 Token, 330
extracted: https://orbit.kiwi/components/utility/ https://orbit.kiwi/components/utility/ 35 Token, 143
extracted: https://orbit.kiwi/development/guides/dictionary/ https://orbit.kiwi/development/guides/dictionary/ 464 Token, 1711
extracted: https://orbit.kiwi/kiwi-use/guides/working-with-figma/ https://orbit.kiwi/kiwi-use/guides/working-with-figma/ 788 Token, 340

extracted: https://orbit.kiwi/components/responsive/hide/ https://orbit.kiwi/components/responsive/hide/ 250 Token, 1107
extracted: https://orbit.kiwi/components/structure/list/ https://orbit.kiwi/components/structure/list/ 498 Token, 2185
extracted: https://orbit.kiwi/components/interaction/tag/ https://orbit.kiwi/components/interaction/tag/ 378 Token, 1640
extracted: https://orbit.kiwi/components/accessibility/skipnavigation/ https://orbit.kiwi/components/accessibility/skipnavigation/ 287 Token, 1146
extracted: https://orbit.kiwi/components/utility/orbitprovider/ https://orbit.kiwi/components/utility/orbitprovider/ 156 Token, 622
extracted: https://orbit.kiwi/kiwi-use/brand/brand-guidelines/ https://orbit.kiwi/kiwi-use/brand/brand-guidelines/ 449 Token, 1772
extracted: https://orbit.kiwi/components/responsive/mobile/ https://orbit.kiwi/components/responsive/mobile/ 276 Token, 1229
extracted: https://orbit.kiwi/components/action/buttonlink/ https://orbit.kiwi/components/action/buttonl

extracted: https://orbit.kiwi/components/interaction/stepper/react/ https://orbit.kiwi/components/interaction/stepper/react/ 937 Token, 3466
extracted: https://orbit.kiwi/components/structure/itinerary/react/ https://orbit.kiwi/components/structure/itinerary/react/ 1223 Token, 4373
extracted: https://orbit.kiwi/getting-started/for-designers/open-source/ https://orbit.kiwi/getting-started/for-designers/open-source/ 252 Token, 954
extracted: https://orbit.kiwi/components/overlay/modal/react/ https://orbit.kiwi/components/overlay/modal/react/ 1603 Token, 5913
extracted: https://orbit.kiwi/foundation/principles/ui-kit-principles/ https://orbit.kiwi/foundation/principles/ui-kit-principles/ 669 Token, 3119
extracted: https://orbit.kiwi/roadmap/ https://orbit.kiwi/roadmap/ 1665 Token, 7655
extracted: https://orbit.kiwi/components/accessibility/skiplink/react/ https://orbit.kiwi/components/accessibility/skiplink/react/ 318 Token, 1243
extracted: https://orbit.kiwi/components/structure/accordio

## Even more dependencies

Install the BERT extractive summarizer and Sentence Transformers, we'll use these to create summaries as a first step.

In [ ]:
!pip install bert-extractive-summarizer sentence-transformers

## Create SBert summaries

We first create SBert summaries by identifying the most central sentences on a page, concatenating those for a rough first summary.

In [80]:
import tiktoken
import jsonlines
from summarizer.sbert import SBertSummarizer

enc = tiktoken.encoding_for_model("text-davinci-003")
model = SBertSummarizer('paraphrase-multilingual-MiniLM-L12-v2')  

with_summaries = []
with jsonlines.open('pages-' + name + '_extracted.jsonl', 'r') as pages:
    for page in pages:
        with_summaries.append(page)
    
print('Creating summaries for ' + str(len(list(with_summaries))) + ' pages.')

for page in with_summaries:
    result = model(page['content']['raw'].replace('\n', ' '), num_sentences=3, min_length=60)
    page['summaries'] = dict()
    page['summaries']['sbert'] = ''.join(result)
    print()
    print(page['url'], page['title'], str(len(enc.encode(page['summaries']['sbert']))) + ' Token,', page['summaries']['sbert'])
        
with jsonlines.open('pages-' + name + '_extracted_summaries.jsonl', 'w') as pages:
    pages.write_all(with_summaries)

print()
print('All summaries created.')

Creating summaries for 253 pages.

https://orbit.kiwi/ https://orbit.kiwi/ 48 Token, Our components are a collection of interface elements that can be reused across the Orbit design system. Color is used to signal structure, highlight importance, and display different states. Visit our community profile and download all of our libraries and resources for free.

https://orbit.kiwi/components https://orbit.kiwi/components 31 Token, Accessibility Components to improve the accessibility of your designs. Action Components enabling users to take actions big and small. Structure Components to structure the information you present.

https://orbit.kiwi/foundation/design-tokens/ https://orbit.kiwi/foundation/design-tokens/ 0 Token, 

https://orbit.kiwi/kiwi-use/content/grammar-and-mechanics/ https://orbit.kiwi/kiwi-use/content/grammar-and-mechanics/ 183 Token, If language isn’t correct, then what’s said isn’t what’s meant; if what’s said isn’t what’s meant, then what must be done remains undone.

/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/pyt


https://orbit.kiwi/component-status/ https://orbit.kiwi/component-status/ 23 Token, Table of contents This listing reflects the current state of our components. The design or documentation could be published before development.

https://orbit.kiwi/kiwi-use/content/voice-and-tone/ https://orbit.kiwi/kiwi-use/content/voice-and-tone/ 83 Token, Voice We speak in a consistent voice so people always know where we’re coming from. We still want to show empathy (and we’re not afraid to say “sorry”), but we have to prioritize letting passengers know how we will deliver on our guarantees and get them where they need to be. Our tone ranges within this spectrum to allow us to express the correct tone for each situation.

https://orbit.kiwi/changelog/ https://orbit.kiwi/changelog/ 117 Token, CountryFlag:The default code for CountryFlag is now "undefined"rather than "anywhere", a gray flag with a question mark. deletes deprecated components, their usage should be replaced with better existing or upc

/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



https://orbit.kiwi/foundation/color/ https://orbit.kiwi/foundation/color/ 66 Token, General guidelines We have different shades for every defined color in Orbit. Use the basic shade as the primary color and only use the additional shades when needed to create more depth or interactions. Product guidelines Use Product Normal for primary actions When used only for primary actions on buttons or text links, the actions tend to stand out from the interface.

https://orbit.kiwi/foundation/spacing https://orbit.kiwi/foundation/spacing 117 Token, Please use only these sizes: |Size name||Value||Examples of use| |xx-small||Great for really small spaces such as connecting an error message to a form.| |x-small||Great to connect related components, like a header and a description.| |small| |medium||Great for inner padding, like left & right padding for content on mobile apps.| |large||Great for left & right inner padding in sections.| |x-large| |xx-large| |xxx-large||Great for space between illust

...


https://orbit.kiwi/components/structure/card/react/ https://orbit.kiwi/components/structure/card/react/ 187 Token, Table of contents Card To implement Card component into your project you’ll need to the import at least the Card and the CardSection: import Card, { CardSection } from "@kiwicom/orbit-components/lib/Card"; After adding import into your project you can use it simply like: Hello World! Props Table below contains all types of the props available in the Card component. Useful when you need different layout then combination of eg | |icon||Display icon on the left| |loading||If | |onClose||Callback that is triggered when Card is closing| |title||The title of the Card| |titleAs||The element used for the root node of the title of Card.| |margin||Utility prop to set margin.| CardSection import Card, { CardSection } from "@kiwicom/orbit-components/lib/Card"; Usage: Hello World!

https://orbit.kiwi/components/interaction/segmentedswitch/react/ https://orbit.kiwi/components/interacti

/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/pyt


https://orbit.kiwi/components/information/alert/react/ https://orbit.kiwi/components/information/alert/react/ 144 Token, Alert To implement Alert component into your project you’ll need to add the import: import Alert from "@kiwicom/orbit-components/lib/Alert"; After adding import into your project you can use it simply like: Hello World! Props The table below contains all types of the props available in Alert component. See Functional specs| |onClose||Function for handling Alert onClose.| |spaceAfter||Additional | |title||The title of the Alert.| type |The type of Alert.| |suppressed||If | enum |type| Functional specs By passing the closableprop into Alert, you will be able to handle onClosefunction and Close icon will be displayed.

https://orbit.kiwi/components/information/badgelist/react/ https://orbit.kiwi/components/information/badgelist/react/ 191 Token, Table of contents BadgeList To implement BadgeList component into your project you’ll need to add the import: import BadgeLis

/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/julrich/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



https://orbit.kiwi/kiwi-use/brand/brand-guidelines/code-kiwi-com/ https://orbit.kiwi/kiwi-use/brand/brand-guidelines/code-kiwi-com/ 63 Token, code.kiwi.com is the brand of the community of people who code and design code.kiwi.com. On our blog, we also write about the things we get up to and the technologies we use. Logo Basic brand resources if you need to promote the code.kiwi.com community.

https://orbit.kiwi/kiwi-use/brand/ https://orbit.kiwi/kiwi-use/brand/ 0 Token, 

https://orbit.kiwi/components/responsive/mobile/react/ https://orbit.kiwi/components/responsive/mobile/react/ 85 Token, Table of contents Mobile To implement Mobile component into your project you’ll need to add the import: import Mobile from "@kiwicom/orbit-components/lib/Mobile"; After adding import into your project you can use it simply like: Hello World! Props Table below contains all types of the props available in the Mobile component. |Name||Type||Description| children |Children components to show.|

https:/

## Extract sections from markdown page content

We'll extract sections from our pages by splitting along markdown headlines (# to ######).

In [83]:
import re
import json
import tiktoken
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from typing import Set
from markdown import markdown
from nltk.tokenize import sent_tokenize

enc = tiktoken.encoding_for_model("text-davinci-003")

def markdown_to_text(markdown_string):
    html = markdown(markdown_string)

    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(enc.encode(text))

def reduce_long(
    long_text: str, long_text_tokens: bool = False, max_len: int = 590
) -> str:
    """
    Reduce a long text to a maximum of `max_len` tokens by potentially cutting at a sentence end
    """
    if not long_text_tokens:
        long_text_tokens = count_tokens(long_text)
    if long_text_tokens > max_len:
        sentences = sent_tokenize(long_text.replace("\n", " "))
        ntokens = 0
        for i, sentence in enumerate(sentences):
            ntokens += 1 + count_tokens(sentence)
            if ntokens > max_len:
                return ". ".join(sentences[:i][:-1]) + "."

    return long_text

discard_categories = []

def extract_sections(
    page_text: str,
    title: str,
    max_len: int = 1500,
    discard_categories: Set[str] = discard_categories,
) -> str:
    """
    Extract the sections of a kickstartDS page, discarding the references and other low information sections
    """
    if len(page_text) == 0:
        return []

    # find all headings and the coresponding contents
    headings = re.findall("#+ .*", page_text)
    for heading in headings:
        page_text = page_text.replace(heading, "#+ !!")
    contents = page_text.split("#+ !!")
    contents = [c.strip() for c in contents]
    assert len(headings) == len(contents) - 1

    cont = contents.pop(0).strip()
    outputs = [(title, "Summary", cont, count_tokens(cont)+4)]
    
    # discard the discard categories, accounting for a tree structure
    max_level = 100
    keep_group_level = max_level
    remove_group_level = max_level
    nheadings, ncontents = [], []
    for heading, content in zip(headings, contents):
        plain_heading = " ".join(heading.split(" ")[1:-1])
        num_equals = len(heading.split(" ")[0])
        if num_equals <= keep_group_level:
            keep_group_level = max_level

        if num_equals > remove_group_level:
            if (
                num_equals <= keep_group_level
            ):
                continue
        keep_group_level = max_level
        if plain_heading in discard_categories:
            remove_group_level = num_equals
            keep_group_level = max_level
            continue
        nheadings.append(heading.replace("#", "").strip())
        ncontents.append(markdown_to_text(content).replace('\n', ' '))
        remove_group_level = max_level

    # count the tokens of each section
    ncontent_ntokens = [
        count_tokens(c)
        + 3
        + count_tokens(" ".join(h.split(" ")[1:-1]))
        - (1 if len(c) == 0 else 0)
        for h, c in zip(nheadings, ncontents)
    ]

    # Create a tuple of (title, section_name, content, number of tokens)
    outputs += [(title, h, c, t) if t<max_len 
                else (title, h, reduce_long(c, max_len), count_tokens(reduce_long(c,max_len))) 
                    for h, c, t in zip(nheadings, ncontents, ncontent_ntokens)]
    
    return outputs

with_sections = []
with jsonlines.open('pages-' + name + '_extracted_summaries.jsonl') as pages:
    for page in pages:
        outputs = []
        outputs += extract_sections(page["content"]["markdown"], page["title"])
        
        df = pd.DataFrame(outputs, columns=["title", "heading", "content", "tokens"])
        df = df[df.tokens>40]
        df = df.drop_duplicates(['title','heading'])
        df = df.reset_index().drop('index',axis=1) # reset index
        df.head()

        result = df.to_json(orient="records")
        parsed = json.loads(result)
        page['sections'] = parsed
        
        with_sections.append(page)

with jsonlines.open('pages-' + name + '_extracted_sections.jsonl', 'w') as pages:
    pages.write_all(with_sections)
    
print('Extracted sections for ' + str(len(with_sections)) + ' pages.')

Extracted sections for 195 pages.
